In [1]:
from foundations.projectables import ProjectableModel, ProjectableIntersection
from foundations.projectables import merge_with_coupling, feedback,feed_forward
from modeling.compute import create_vars
import numpy as np
from numpy import pi

In [2]:
A,V,S,W,Ww,CL,Re,Cf,CD,D = create_vars('A V S W W_w C_L Re C_f C_D D')
rho,mu,k,t,e,CDA,SwetS,W0 = 1.23, 1.78e-5, 1.2, 0.12, 0.96, 0.0306, 2.05, 4940

Aero = ProjectableModel()
CL = Aero.Var('C_L', 2*W/(rho*V**2*S))
Re = Aero.Var('Re', rho*V/mu*(S/A)**(1/2))
Cf = Aero.Var('C_f', 0.074/Re**0.2)
CD = Aero.Var('C_D', CDA/S+k*Cf*SwetS+CL**2/(pi*A*e))
D  = Aero.Var('D', 0.5*rho*V**2*CD*S)
Struct = ProjectableModel()
Ww = Struct.Var('Ww', 45.42*S+8.71e-5*2.5/t*A**(3/2)*S**(1/2)*(W0*W)**(1/2))
Struct.add_equation(W, W0+Ww)
Struct.solvepar(x_initial={W:1000})
AeroStruct = merge_with_coupling(Struct, Aero)

# Test single discipline

### Structures

In [3]:
F_S = Struct.functional()
y = {A:10, S:10}
z = F_S.dict_in_dict_out(y, cleanup=True)
{key: np.round(value, 2) for key, value in z.items()}

{W: 6415.76, Ww: 1475.76}

### Aero

In [4]:
F_A = Aero.functional()
y = {A:10, S:10, V:10, W:6400}
F_A.dict_in_dict_out(y, cleanup=True)

{C_L: 10.406504065040641,
 Re: 691011.235955056,
 C_f: 0.005027298793072368,
 C_D: 3.6062056097889377,
 D: 2217.8164500201965}

# Test multiple disciplines

### Based on shared variable couplings

In [6]:
F_coupling = AeroStruct.functional(x_initial={W:1000})
y = {A:10, S:10, V:10}
F_coupling.dict_in_dict_out(y, cleanup=True)

{W: 6415.7614501678045}

### Based on feedback

In [24]:
AeroStruct_default = ProjectableIntersection(Struct, Aero)
F_feedback = AeroStruct_default.functional()
F_feedback.dict_in_dict_out(y, cleanup=True)

{W: 6415.7614501678045,
 Ww: 1475.761450167805,
 C_L: 10.432132439297233,
 Re: 691011.235955056,
 C_f: 0.005027298793072368,
 C_D: 3.6239135990896294,
 D: 2228.706863440122}